In [1]:
import pandas as pd
import numpy as np
import sys
import matplotlib.pyplot as plt
from statsmodels.nonparametric.smoothers_lowess import lowess
from datetime import timedelta, datetime
from pykalman import KalmanFilter

In [6]:
cpu_data = pd.read_csv('sysinfo.csv')
print(type(cpu_data['timestamp']))
print(cpu_data['timestamp'])

<class 'pandas.core.series.Series'>
0       2018-05-10 00:00:34.102496
1       2018-05-10 00:01:34.106931
2       2018-05-10 00:02:34.111049
3       2018-05-10 00:03:34.115387
4       2018-05-10 00:04:34.119495
5       2018-05-10 00:05:34.123510
6       2018-05-10 00:06:34.127105
7       2018-05-10 00:07:34.130987
8       2018-05-10 00:08:34.134975
9       2018-05-10 00:09:34.139116
10      2018-05-10 00:10:34.143925
11      2018-05-10 00:11:34.148410
12      2018-05-10 00:12:34.152373
13      2018-05-10 00:13:34.156271
14      2018-05-10 00:14:34.160362
15      2018-05-10 00:15:34.164712
16      2018-05-10 00:16:34.168885
17      2018-05-10 00:17:34.172822
18      2018-05-10 00:18:34.177300
19      2018-05-10 00:19:34.181473
20      2018-05-10 00:20:34.188217
21      2018-05-10 00:21:34.192335
22      2018-05-10 00:22:34.196461
23      2018-05-10 00:23:34.200561
24      2018-05-10 00:24:34.204558
25      2018-05-10 00:25:34.208543
26      2018-05-10 00:26:34.212714
27      2018-05-10 

In [8]:
cpu_data['timestamp'] = pd.to_datetime(cpu_data['timestamp'])
print(type(cpu_data['timestamp']))
print(cpu_data['timestamp'])

<class 'pandas.core.series.Series'>
0      2018-05-10 00:00:34.102496
1      2018-05-10 00:01:34.106931
2      2018-05-10 00:02:34.111049
3      2018-05-10 00:03:34.115387
4      2018-05-10 00:04:34.119495
5      2018-05-10 00:05:34.123510
6      2018-05-10 00:06:34.127105
7      2018-05-10 00:07:34.130987
8      2018-05-10 00:08:34.134975
9      2018-05-10 00:09:34.139116
10     2018-05-10 00:10:34.143925
11     2018-05-10 00:11:34.148410
12     2018-05-10 00:12:34.152373
13     2018-05-10 00:13:34.156271
14     2018-05-10 00:14:34.160362
15     2018-05-10 00:15:34.164712
16     2018-05-10 00:16:34.168885
17     2018-05-10 00:17:34.172822
18     2018-05-10 00:18:34.177300
19     2018-05-10 00:19:34.181473
20     2018-05-10 00:20:34.188217
21     2018-05-10 00:21:34.192335
22     2018-05-10 00:22:34.196461
23     2018-05-10 00:23:34.200561
24     2018-05-10 00:24:34.204558
25     2018-05-10 00:25:34.208543
26     2018-05-10 00:26:34.212714
27     2018-05-10 00:27:34.216949
28     2018-

In [9]:
cpu_data['timestamp'] = pd.to_timedelta(cpu_data['timestamp'])
print(type(cpu_data['timestamp']))
print(cpu_data['timestamp'])

<class 'pandas.core.series.Series'>
0      17661 days 00:00:34.102496
1      17661 days 00:01:34.106931
2      17661 days 00:02:34.111049
3      17661 days 00:03:34.115387
4      17661 days 00:04:34.119495
5      17661 days 00:05:34.123510
6      17661 days 00:06:34.127105
7      17661 days 00:07:34.130987
8      17661 days 00:08:34.134975
9      17661 days 00:09:34.139116
10     17661 days 00:10:34.143925
11     17661 days 00:11:34.148410
12     17661 days 00:12:34.152373
13     17661 days 00:13:34.156271
14     17661 days 00:14:34.160362
15     17661 days 00:15:34.164712
16     17661 days 00:16:34.168885
17     17661 days 00:17:34.172822
18     17661 days 00:18:34.177300
19     17661 days 00:19:34.181473
20     17661 days 00:20:34.188217
21     17661 days 00:21:34.192335
22     17661 days 00:22:34.196461
23     17661 days 00:23:34.200561
24     17661 days 00:24:34.204558
25     17661 days 00:25:34.208543
26     17661 days 00:26:34.212714
27     17661 days 00:27:34.216949
28     17661

In [11]:
cpu_data['plottable_timestamp'] = cpu_data['timestamp'].apply(timedelta.total_seconds)
cpu_data

,cpu_freq,cpu_percent,sys_load_1,temperature,timestamp,plottable_timestamp
0,1652.985500,0.0819,0.67,38,17661 days 00:00:34.102496,1.525910e+09
1,1751.238375,0.0746,0.59,38,17661 days 00:01:34.106931,1.525910e+09
2,1644.140375,0.0564,0.60,40,17661 days 00:02:34.111049,1.525911e+09
3,1827.442625,0.0925,0.61,40,17661 days 00:03:34.115387,1.525911e+09
4,1797.277125,0.0794,0.46,39,17661 days 00:04:34.119495,1.525911e+09
5,2115.837125,0.1050,1.03,39,17661 days 00:05:34.123510,1.525911e+09
6,1674.788125,0.0746,1.68,40,17661 days 00:06:34.127105,1.525911e+09
7,1616.682750,0.0324,1.16,38,17661 days 00:07:34.130987,1.525911e+09
8,1962.316500,0.1221,1.45,40,17661 days 00:08:34.134975,1.525911e+09
9,3340.498625,0.1264,0.93,43,17661 days 00:09:34.139116,1.525911e+09
